In [1]:
### Init network Build
import numpy as np
from pathlib import Path
import pandas as pd
from osgeo import gdal
import keras
import time
import tensorflow as tf
print(tf.__version__, 'tf')
print(keras.__version__, 'keras')

2.3.0 tf
2.4.3 keras


In [2]:
df_merged = pd.read_csv('image_metrics_030621.csv')

In [3]:
print(df_merged.shape)
df_merged.head()

(87647, 11)


,Unnamed: 0,img_res,img_type,img_index,da_num,img_path,da_number,cwa_determination,metrics,mean,stdev
0,0,lores,mndwi,292,LRC-2003-22227,/data/image_final2/GEE_images_final2_lores_mnd...,LRC-2003-22227,1,"(-0.3306894463763968, 0.09898692070930509)",-0.330689,0.098987
1,1,hires,mndwi,8157,SAC-2007-01367,/data/image_final2/GEE_images_final2_hires_mnd...,SAC-2007-01367,0,"(-0.40988840562690426, 0.19788100896017022)",-0.409888,0.197881
2,2,lores,srtm,5495,MVP-2020-00252-MHK,/data/image_final2/GEE_images_final2_lores_srt...,MVP-2020-00252-MHK,0,"(211.5080108642578, 18.443937482788826)",211.508011,18.443937
3,3,lores,gmndwi,5824,NAN-2016-00095-USH,/data/image_final2/GEE_images_final2_lores_gmn...,NAN-2016-00095-USH,0,"(0.56878513, 0.23097283)",0.568785,0.230973
4,4,hires,srtm,1436,LRL-2016-00917-htm,/data/image_final2/GEE_images_final2_hires_srt...,LRL-2016-00917-htm,0,"(266.44921875, 1.7671324110003224)",266.449219,1.767132


In [4]:
f3 = df_merged.da_num.apply(lambda x: x[:3])
f3.value_counts()

MVP    14375
MVN     8057
SWG     7395
NAO     6484
MVK     5909
LRC     4946
NWO     4676
SAC     4459
POA     4452
LRH     3148
LRL     3012
LRB     2889
NWK     2732
LRE     1742
SWL     1700
SAW     1687
NAN     1674
MVM     1499
NAB     1400
NWP     1023
NWS     1020
NAP      940
MVR      790
SWT      428
POH      340
LRP      325
NAE      290
NWW      160
MVS       50
LRN       45
Name: da_num, dtype: int64

In [5]:
dir2 = '/data/training_data2/'
train_X = [f for f in Path(dir2).iterdir() if 'MVK' in f.name]

In [6]:
train_X[0].name[:-4]

'MVK-2016-00412-TB'

In [7]:
def get_label(da_num_i, df_merged = df_merged):
    return int(df_merged[df_merged.da_num == da_num_i].cwa_determination.iloc[0])
get_label(train_X[0].name[:-4])

0

In [8]:
train_y = [get_label(x.name[:-4]) for x in train_X]

In [9]:
print(len(train_X), len(train_y))

585 585


In [10]:
arr_train_X = np.array([np.load(f) for f in train_X])

In [11]:
arr_train_X.shape

(585, 9, 256, 256)

In [15]:
!nvidia-smi

Tue Mar  9 02:58:50 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           On   | 00000000:00:1E.0 Off |                    0 |
| N/A   26C    P0    69W / 149W |  10767MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [14]:
from keras.applications.vgg16 import VGG16

vgg16_model1 = VGG16(include_top=False, weights='imagenet', input_shape=(256,256, 3))
vgg16_model2 = VGG16(include_top=False, weights='imagenet', input_shape=(256,256, 3))
vgg16_model3 = VGG16(include_top=False, weights='imagenet', input_shape=(256,256, 3))

vgg16_model1.trainable = False
vgg16_model2.trainable = False
vgg16_model3.trainable = False

58892288/58889256 [==============================] - 1s 0us/step


In [16]:
from keras.layers import Dense, Concatenate, Input, Lambda, Flatten, Softmax
from keras.models import Model, Sequential

num_channels = 9
input = Input(shape=(256,256, num_channels))

branch_outputs = []

# loop for however many branches you want to use.  (each branch needs 3 channel depth)
for i in [0,3,6]:
    # create looping so that vgg16 model input is created ie 256,256,3
    print(input.shape)
    out = Lambda(lambda x: x[:,:,:,i:i+3])(input)
    print(out.shape)
    
    # Setting up your layers in each branch: (currently each branch is identical architecture.)
    if i == 0: out = Sequential(vgg16_model1)(out)   # use pretrained and loaded vgg16
    elif i == 3: out = Sequential(vgg16_model2)(out)
    else: out = Sequential(vgg16_model3)(out)   # use pretrained and loaded vgg16
    print(out.shape)
    out = Dense(10)(out)
    print(out.shape)
    out = Flatten()(out)
    print(out.shape)
    branch_outputs.append(out)

# Concatenating the branches outputs:
out = Concatenate()(branch_outputs)
print(out.shape)

# Add final dense layers and softmax
out = Dense(10)(out)
print(out.shape)
out = Dense(6)(out)
out = Dense(2)(out)
print(out.shape)
out = Softmax()(out)
print(out.shape)

(None, 256, 256, 9)
(None, 256, 256, 3)
(None, 8, 8, 512)
(None, 8, 8, 10)
(None, 640)
(None, 256, 256, 9)
(None, 256, 256, 3)
(None, 8, 8, 512)
(None, 8, 8, 10)
(None, 640)
(None, 256, 256, 9)
(None, 256, 256, 3)
(None, 8, 8, 512)
(None, 8, 8, 10)
(None, 640)
(None, 1920)
(None, 10)
(None, 2)
(None, 2)


In [17]:
model = Model(inputs=input, outputs=out)   
model.summary()
#plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 256, 256, 9) 0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 256, 256, 3)  0           input_5[0][0]                    
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 256, 256, 3)  0           input_5[0][0]                    
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 256, 256, 3)  0           input_5[0][0]                    
_______________________________________________________________________________________

In [18]:
model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
t0 = time.time()
model.fit(arr_train_X, train_y, epochs=3, batch_size=64, validation_split = 0.2, workers=4, use_multiprocessing=True)
print("Train time:", time.time()-t0)